This notebook implements Implicit Q-Learning to solve Maze 2D.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
from loguru import logger
import matplotlib.pyplot as plt
from IPython.display import Image
'''from src.utils import (
    get_device,
    set_seed,
    eval_policy,
    demo_policy,
    plot_returns,
    save_frames_as_gif,
    return_range
)'''
#from tqdm import tqdm
#import einops
import os
import copy

from src.d4rl_dataset import D4RLSampler

plt.ion()

We start by importing D4RL, a library that contains training data obtained by running policy of different levels on a few environments.

In [6]:
import minari

dataset = minari.load_dataset("NathanGavenski/CartPole-v1", download=True)

for episode_data in dataset.iterate_episodes():
    observations = episode_data.observations
    actions = episode_data.actions
    rewards = episode_data.rewards
    terminations = episode_data.terminations
    truncations = episode_data.truncations
    infos = episode_data.infos

ValueError: Couldn't find any version for dataset NathanGavenski/CartPole in the remote Farama server.

In [ ]:
SEED: int = 42
ENVIRONMENT_NAME: str='halfcheetah-medium-replay-v2'

# torch related defaults
DEVICE = get_device()
torch.set_default_dtype(torch.float32)

# Use random seeds for reproducibility
set_seed(SEED)

# instantiate the environment
env = gym.make(ENVIRONMENT_NAME)

# get the state and action dimensions
action_dimension = env.action_space.shape[0]
state_dimension = env.observation_space.shape[0]

logger.info(f'Action Dimension: {action_dimension}')
logger.info(f'Action High: {env.action_space.high}')
logger.info(f'Action Low: {env.action_space.low}')
logger.info(f'State Dimension: {state_dimension}')